In [1]:
import sys
import pprint

sys.setrecursionlimit(10000)  # if needed
pp = pprint.PrettyPrinter(depth=6, width=120)
# pp.pprint(large_object)

In [2]:
import re

def parse_facts(file_path):
    facts = {}
    target = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("target:"):
                target_line = line[len("target:"):].strip()
                key, op, val = parse_condition(target_line)
                target = (key, val)  # Only store key and expected value
                continue
            if '=' in line:
                key, val = line.split('=')
                try:
                    facts[key.strip()] = int(val.strip())
                except:
                    facts[key.strip()] = val.strip()
            elif 'is' in line:
                key, val = line.split('is')
                facts[key.strip()] = val.strip()
            else:
                facts[line.strip()] = True
    return facts, target

In [3]:
def parse_condition(cond):
    cond = cond.strip()
    if '>=' in cond:
        key, val = cond.split('>=')
        return (key.strip(), '>=', float(val.strip()))
    elif '<=' in cond:
        key, val = cond.split('<=')
        return (key.strip(), '<=', float(val.strip()))
    if '>' in cond:
        key, val = cond.split('>')
        return (key.strip(), '>', float(val.strip()))
    elif '<' in cond:
        key, val = cond.split('<')
        return (key.strip(), '<', float(val.strip()))
    elif '=' in cond:
        key, val = cond.split('=')
        try:
            return (key.strip(), '=', int(val.strip()))
        except:
            return (key.strip(), '=', val.strip())
    elif 'is' in cond:
        key, val = cond.split('is')
        return (key.strip(), 'is', val.strip())
    else:
        return (cond.strip(), '==', True)


In [4]:
def parse_rules(file_path):
    rules = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or 'IF' not in line or 'THEN' not in line:
                continue
            cond_part, conclusion = line.split('THEN')
            cond_part = cond_part.replace('IF', '').strip()
            conclusion = conclusion.strip()

            and_conditions = [c.strip() for c in re.split(r'\bAND\b', cond_part)]
            condition_list = []
            for cond in and_conditions:
                if 'OR' in cond:
                    or_parts = [c.strip() for c in cond.split('OR')]
                    or_group = [parse_condition(c) for c in or_parts]
                    condition_list.append(('OR', or_group))
                else:
                    condition_list.append(parse_condition(cond))
            conclusion_parsed = parse_condition(conclusion)
            rules.append((condition_list, conclusion_parsed))
    return rules


In [5]:
def evaluate_condition(facts, cond):
    key, op, val = cond
    if key not in facts:
        return False
    try:
        if op == '=' or op == 'is':
            return facts[key] == val
        elif op == '==':
            return bool(facts[key])
        elif op == '>':
            return float(facts[key]) > float(val)
        elif op == '<':
            return float(facts[key]) < float(val)
        elif op == '>=':
            return float(facts[key]) >= float(val)
        elif op == '<=':
            return float(facts[key]) <= float(val)
    except:
        return False
    return False


In [6]:
def evaluate_conditions(facts, conditions):
    for cond in conditions:
        if isinstance(cond, tuple) and cond[0] == 'OR':
            if not any(evaluate_condition(facts, c) for c in cond[1]):
                return False
        else:
            if not evaluate_condition(facts, cond):
                return False
    return True


In [7]:
def forward_chaining(facts, rules, target=None):
    changed = True
    while changed:
        changed = False
        print("Current facts:", facts)
        for conditions, conclusion in rules:
            key, op, val = conclusion
            if key in facts and facts[key] == val:
                continue
            if evaluate_conditions(facts, conditions):
                facts[key] = val
                changed = True
                if target and key == target[0] and facts[key] == target[1]:
                    print(f"\nTarget reached: {key} ")
                    return facts
    return facts


In [8]:
facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [9]:
print("\n--- Forward Chaining ---")
final_facts = forward_chaining(facts, rules, target)

print("\nFinal inferred facts:")
for fact in final_facts:
    print(f"{fact} = {final_facts[fact]}")


--- Forward Chaining ---
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}

Target reached: citrus_fruit 

Final inferred facts:
seeds = 0
diameter = 7
skin_smell = True
color = orange
perfumed = True
size = medium
fruit = orange
citrus_fruit = True


In [10]:
def backward_chaining(facts, rules, target, depth=0, visited=None):
    """
    Backward chaining algorithm compatible with existing functions

    Args:
        facts: Dictionary of known facts
        rules: List of rules in the format [(conditions, conclusion), ...]
        target: Tuple (key, value) representing the target to prove
        depth: Current recursion depth (for indentation)
        visited: Set of visited targets to prevent infinite recursion

    Returns:
        Boolean indicating if target was proven
    """
    indent = "  " * depth
    target_key, target_val = target

    print(f"{indent}Trying to prove: {target_key} = {target_val}")

    # Check if target is already in facts
    if target_key in facts:
        if facts[target_key] == target_val:
            print(f"{indent}✓ Already known: {target_key} = {facts[target_key]}")
            return True
        else:
            print(f"{indent}× Known but with different value: {target_key} = {facts[target_key]} (needed {target_val})")
            return False

    # Initialize visited set to prevent infinite recursion
    if visited is None:
        visited = set()

    # Prevent infinite recursion
    target_repr = f"{target_key}={target_val}"
    if target_repr in visited:
        print(f"{indent}× Already visited: {target_repr} (avoiding loop)")
        return False

    visited.add(target_repr)
    print(f"{indent}Searching rules for conclusion: {target_key}")

    # Find rules that have our target in the conclusion
    relevant_rules = []
    for i, (conditions, conclusion) in enumerate(rules):
        conclusion_key, conclusion_op, conclusion_val = conclusion

        # Check if this rule concludes our target
        if conclusion_key == target_key and conclusion_val == target_val:
            relevant_rules.append((i, conditions, conclusion))

    # Try each relevant rule
    for rule_index, conditions, conclusion in relevant_rules:
        conclusion_key, conclusion_op, conclusion_val = conclusion

        print(f"{indent}Found rule {rule_index+1} with conclusion: {conclusion_key} {conclusion_op} {conclusion_val}")

        # Print the conditions we need to check
        print(f"{indent}Checking conditions:")
        conditions_met = True

        # Check each condition
        for condition in conditions:
            if isinstance(condition, tuple) and condition[0] == 'OR':
                # Handle OR conditions
                print(f"{indent}  Checking OR conditions:")
                or_conditions = condition[1]
                or_satisfied = False

                for or_cond in or_conditions:
                    cond_key, cond_op, cond_val = or_cond
                    print(f"{indent}    Checking: {cond_key} {cond_op} {cond_val}")

                    if cond_key in facts:
                        # Condition is already in facts, check if it's satisfied
                        if evaluate_condition(facts, or_cond):
                            print(f"{indent}    ✓ Condition already satisfied: {cond_key} = {facts[cond_key]}")
                            or_satisfied = True
                            break
                        else:
                            print(f"{indent}    × Condition not satisfied: {cond_key} = {facts[cond_key]}")
                    else:
                        # Try to prove this condition recursively
                        print(f"{indent}    Need to prove: {cond_key} = {cond_val}")
                        if backward_chaining(facts, rules, (cond_key, cond_val), depth+1, visited.copy()):
                            print(f"{indent}    ✓ Successfully proved: {cond_key} = {facts[cond_key]}")
                            or_satisfied = True
                            break
                        else:
                            print(f"{indent}    × Failed to prove: {cond_key} = {cond_val}")

                if not or_satisfied:
                    print(f"{indent}  × No OR conditions were satisfied")
                    conditions_met = False
                    break
                else:
                    print(f"{indent}  ✓ OR condition satisfied")
            else:
                # Handle regular condition
                cond_key, cond_op, cond_val = condition
                print(f"{indent}  Checking: {cond_key} {cond_op} {cond_val}")

                if cond_key in facts:
                    # Condition is already in facts, check if it's satisfied
                    if evaluate_condition(facts, condition):
                        print(f"{indent}  ✓ Condition already satisfied: {cond_key} = {facts[cond_key]}")
                    else:
                        print(f"{indent}  × Condition not satisfied: {cond_key} = {facts[cond_key]}")
                        conditions_met = False
                        break
                else:
                    # Try to prove this condition recursively
                    print(f"{indent}  Need to prove: {cond_key} = {cond_val}")
                    if not backward_chaining(facts, rules, (cond_key, cond_val), depth+1, visited.copy()):
                        print(f"{indent}  × Failed to prove: {cond_key} = {cond_val}")
                        conditions_met = False
                        break
                    else:
                        print(f"{indent}  ✓ Successfully proved: {cond_key} = {facts[cond_key]}")

        # If all conditions are met, add the conclusion to facts
        if conditions_met:
            facts[conclusion_key] = conclusion_val
            print(f"{indent}✓ Rule fired! Added: {conclusion_key} = {conclusion_val}")
            return True

    print(f"{indent}× Could not prove: {target_key} = {target_val}")
    return False

In [11]:
# Parse facts and rules using your existing functions
facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')
print("Initial facts:")
# for key, value in facts.items():
#     print(f"{key} = {value}")
# print(f"\nTarget to prove: {target[0]} = {target[1]}\n")
# # Print the parsed rules for clarity
# print("Rules:")
# for i, (conditions, conclusion) in enumerate(rules):
#     cond_str = []
#     for c in conditions:
#         if isinstance(c, tuple) and c[0] == 'OR':
#             or_parts = [f"{key} {op} {val}" for key, op, val in c[1]]
#             cond_str.append(f"({' OR '.join(or_parts)})")
#         else:
#             key, op, val = c
#             cond_str.append(f"{key} {op} {val}")
#     key, op, val = conclusion
#     print(f"Rule {i+1}: IF {' AND '.join(cond_str)} THEN {key} {op} {val}")
print("\n--- Backward Chaining ---")
# Don't use facts.copy() here - use the original facts dictionary
result = backward_chaining(facts, rules, (target[0], target[1]))
print(f"\nTarget '{target[0]} = {target[1]}' inferred: {result}")
print("\nFinal facts after inference:")
for key, value in sorted(facts.items()):
    print(f"{key} = {value}")

Initial facts:

--- Backward Chaining ---
Trying to prove: citrus_fruit = True
Searching rules for conclusion: citrus_fruit
Found rule 5 with conclusion: citrus_fruit == True
Checking conditions:
  Checking OR conditions:
    Checking: fruit is lemon
    Need to prove: fruit = lemon
  Trying to prove: fruit = lemon
  Searching rules for conclusion: fruit
  Found rule 6 with conclusion: fruit is lemon
  Checking conditions:
    Checking: size is medium
    Need to prove: size = medium
    Trying to prove: size = medium
    Searching rules for conclusion: size
    Found rule 13 with conclusion: size is medium
    Checking conditions:
      Checking: diameter > 2.0
      ✓ Condition already satisfied: diameter = 7
      Checking: diameter < 10.0
      ✓ Condition already satisfied: diameter = 7
    ✓ Rule fired! Added: size = medium
    ✓ Successfully proved: size = medium
    Checking: color is yellow
    × Condition not satisfied: color = orange
  × Could not prove: fruit = lemon
    × 